In [ ]:
#Install and Import Necessary Tools#

pip install selenium
pip install chromedriver
pip install webdriver_manager 

In [1]:
import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

In [2]:
##Read in CSV containing Guides##

guides = pd.read_csv('Guides_Start_Worked_Test_1.csv', 
            index_col='guide', 
            header=0, 
            names=['guide', 'Gene ID', 'sequence', ''])
print(guides)

         Gene ID              sequence      
guide                                       
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN
...          ...                   ...   ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN

[81 rows x 3 columns]


In [7]:
## WORKING FULL 1 - With Loop ##

def get_genomic_location():
    
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.common.exceptions import NoSuchElementException
    import time

    driver = webdriver.Chrome(ChromeDriverManager().install()) #define driver, I use Chrome, then load url
    driver.get("https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg38&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=chr7%3A5528404%2D5530403&hgsid=815674809_ypAZM1pVZHGgvsosmAcenVUFkPzy")
    assert "Human hg38 chr7:5528404-5530403 UCSC Genome Browser v395" in driver.title
    driver.minimize_window()

    chrsm_loc = [] #empty list where genomic locations will go into
    
    for i in guides["sequence"]: #feed in the guide from the dataframe through the loop
        
        time.sleep(5)
        elem = driver.find_element_by_name("hgt.positionInput") #find search bar
        elem.clear() #clear search bar
        elem.send_keys(i) #enter guide
        elem.send_keys(Keys.RETURN) #enter aka click
        try: 
            browser = driver.find_element_by_partial_link_text('browser') #return to genome browser
            browser.send_keys(Keys.RETURN)
            
            zoom = driver.find_element_by_name("hgt.out4") #zoom out by 100x for 2,000bp
            zoom.send_keys(Keys.RETURN)
            
            time.sleep(12)
            
            chrsm = driver.find_element_by_id("positionDisplay").text #take genomic position as text
            #print(chrsm)
            chrsm_loc.append(chrsm) #add it to the empty list
            loc = pd.DataFrame(chrsm_loc) #make it into a data frame in order to add to initial csv -> dataframe
            guides["Genomic Location"] = loc
            print(guides)

        except NoSuchElementException:  #this is for guides that do not return a match from UCSC genome browser
            chrsm = "NA" #enter NA for failed ones and append to that guide in the dataframe
            chrsm_loc.append(chrsm)
            loc = pd.DataFrame(chrsm_loc)
            guides["Genomic Location"] = loc
            print(guides)
            pass
            driver.back() #go back one page to start over
            continue
        
    driver.close()
    
   


In [11]:
## FULL 1 ALONE ##

get_genomic_location()
guides_drop = guides.drop([0]) #drop the SHAM row to keep going
guides_new = guides_drop
print(guides_new)


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
         Gene ID              sequence        Genomic Location
guide                                                         
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM               NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN              NaN
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN              NaN
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN              NaN
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN              NaN
...          ...                   ...    ...              ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN              NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA              NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN              NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN              NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN              NaN

[85 r

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN                           NaN
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA                           NaN
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

[85 rows x 4 columns]
         Gene ID              sequence                     Genomic Location
guide                                                                      
0           SHAM  CTTTCCTTCCCAGGGCGTGA   SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA    NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA    NaN    chr7:26,196,703-26,198,702
3          IKZF1  TTCTTCTTTCTCATCAGGGA    NaN                            NA
4           SND1  CCCCTACGTAGGTCCTCTCA    NaN  chr7:127,685,612-127,687,611
...          ...                   ...    ...                           ...
80      GTF2IRD1  TTGGACAGCGCTGAGCACTG    NaN    chr7:74,511,837-74,513,836
81          ISPD  CGCCCGCAGCCATGGAGGCC  CRPPA    chr7:16,420,324-16,422,323
82           ZP3  TGCTGTTCTTCTCTCAGGCA    NaN                           NaN
83         CALN1  CCAGAACCCGAAAGGCCTCT    NaN                           NaN
84        LSMEM1  TTGTTTCATATTAGCTCTAG    NaN                     

In [12]:
###create doc with gene_loc full, no SHAM###

# guides_new.to_csv("Guides_With_Genomic_Locations_Test_1", sep = '\t')

In [5]:
## WORKING FULL 2 - LOOP TEST ##

def get_primer_pairs():

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.support.ui import Select
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    import time
    from selenium.common.exceptions import NoSuchElementException
    
    pair_1_seq = [] #empty lists for elements to go into
    pair_2_seq = []
    forward_Tm = []
    reverse_Tm = []
    product_length = []

    for j in guides_new["Genomic Location"]: #feed in the genomic location from the dataframe through the loop

        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get("https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?id=GCF_000001405.39")
        driver.minimize_window()
        #assert "NCBI's" in driver.title
        
        time.sleep(3)

        elem = driver.find_element_by_id("loc-search") #find search bar
        elem.clear() #clear search bar

        elem.send_keys(j) #enter genomic location via j
        elem.send_keys(Keys.RETURN)


        wait = WebDriverWait(driver, 100) #create waitime to allow page to load

        #primer blast via "Tools"
        tools = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-qtip='Tools']"))) #select Tools dynamic id
#         time.sleep(5) #slows down after ~25, if less no need for this pause
        tools.click()

        text1 = "BLAST and Primer Search";
        drop1 = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='" +text1 +"']"))) #select option from dropdown
        drop1.click()

        text2 = "Primer BLAST (Visible Range)";
        drop2 = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='" +text2 +"']"))) #select option from dropdown
        drop2.click() #this will redirect to Primer BLAST page

        driver.minimize_window()

        #switch windows to new window
        window_before = driver.window_handles[0] #starting window set
        window_after = driver.window_handles[1] #new window set

        driver.switch_to.window(window_after) #switch to new window to be able to intercat with it
        print(driver.current_url) #check window switch successful

        #set PCR product sizes
        primemin = driver.find_element_by_name("PRIMER_PRODUCT_MIN") #find search bar
        primemin.clear() #clear search bar
        primemin.send_keys("500") #enter genomic location

        primemax = driver.find_element_by_name("PRIMER_PRODUCT_MAX") #find search bar
        primemax.clear() #clear search bar
        primemax.send_keys("1000") #enter genomic location


        ###find genomic positions to make 200bp around cut site###
        forward_from = driver.find_element_by_id("PRIMER5_START").get_attribute("value") #value from gen browser

        forward_to = driver.find_element_by_id("PRIMER5_END") #will input new value here

        forward_to_num = int(forward_from) + 900 #this adjusts to create 200bp buffer zone around CRISPR cut site
        forward_to.send_keys(forward_to_num)

        ##reverse##
        reverse_to = driver.find_element_by_id("PRIMER3_END").get_attribute("value")

        reverse_from = driver.find_element_by_id("PRIMER3_START") #will input new value here

        reverse_from_num = int(reverse_to) - 900 #this adjusts to create 200bp buffer zone around CRISPR cut site
        reverse_from.send_keys(reverse_from_num)



        #Select reference data base
        database= Select(driver.find_element_by_id('PRIMER_SPECIFICITY_DATABASE'))
        database.select_by_index(2) #"Genomes for selected organisms (primary reference assembly only)"

        #Primer BLAST
        get_primers = driver.find_element_by_id("b1").click()
        driver.minimize_window()
        #time.sleep(5)
        
        time.sleep(120) # this sleep time is critical to allow website to load while finding primers
       

        pair_1 = driver.find_element_by_css_selector(
        "#alignments > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(2)"
        ).get_attribute("innerHTML") #get text Forward Primer
        #print(pair_1)

        pair_1_seq.append(pair_1) #add to empty list

        seq_1 = pd.DataFrame(pair_1_seq) #convert to data.Frame
        guides["Pair 1"] = seq_1 #add to csv


        ##reverse##
        pair_2 = driver.find_element_by_css_selector(
        "#alignments > div:nth-child(1) > table > tbody > tr:nth-child(3) > td:nth-child(2)"
        ).get_attribute("innerHTML") #get text Reverse Primer
        #print(pair_2)
        pair_2_seq.append(pair_2)

        seq_2 = pd.DataFrame(pair_2_seq)
        guides["Pair 2"] = seq_2

        ##forward Tm##
        pair_1_Tm = driver.find_element_by_css_selector(
        "#alignments > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(7)"
        ).get_attribute("innerHTML") #get text Forward Tm

        #print(pair_1_Tm)
        forward_Tm.append(pair_1_Tm)

        Tm_1 = pd.DataFrame(forward_Tm)

        guides["Forward Tm"] = Tm_1

        ##reverse Tm##
        pair_2_Tm = driver.find_element_by_css_selector(
        "#alignments > div:nth-child(1) > table > tbody > tr:nth-child(3) > td:nth-child(7)"
        ).get_attribute("innerHTML") #get text Reverse Tm

        #print(pair_2_Tm)
        reverse_Tm.append(pair_2_Tm)

        Tm_2 = pd.DataFrame(reverse_Tm)

        guides["Reverse Tm"] = Tm_2

        ##product length##
        product_len = driver.find_element_by_css_selector(
        "#alignments > div:nth-child(1) > table > tbody > tr:nth-child(5) > td"
        ).get_attribute("innerHTML") #get text Product Length

        #print(product_len)
        product_length.append(product_len)

        len = pd.DataFrame(product_length)

        guides["Product Length"] = len

        driver.quit()

    print(guides)

    guides.to_csv("guides_full_test_1", sep = '\t')





In [26]:
## FULL 2 ALONE ##

get_primer_pairs()
print(guides_new)
guides_final = guides_new
print(guides_final)


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
https://www.ncbi.nlm.nih.gov/tools/primer-blast/index.cgi?ORGANISM=Homo%20sapiens&INPUT_SEQUENCE=NC_000007.14&PRIMER5_START=5528404&PRIMER3_END=5530403&PRIMER_PRODUCT_MAX=1999&SHOW_SVIEWER=true

Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
https://www.ncbi.nlm.nih.gov/tools/primer-blast/index.cgi?ORGANISM=Homo%20sapiens&INPUT_SEQUENCE=NC_000007.14&PRIMER5_START=26196703&PRIMER3_END=26198702&PRIMER_PRODUCT_MAX=1999&SHOW_SVIEWER=true
         Gene ID              sequence                       Genomic Location  \
guide                                                                           
0           SHAM  CTTTCCTTCCCAGGGCGTGA       SHAM                          NA   
1           ACTB 

In [8]:
## BOTH FUNCTIONS ##

get_genomic_location()
guides_drop = guides.drop([0])
guides_new = guides_drop
# guides_new = guides
print(guides_new)
    
get_primer_pairs()
guides_final = guides_new
print(guides_final)


guides_final.to_csv("Guides_With_Genomic_Locations_and_Primers_Final", sep = '\t')


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
         Gene ID              sequence       Genomic Location
guide                                                        
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM               NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN              NaN
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN              NaN
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN              NaN
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN              NaN
...          ...                   ...   ...              ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN              NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN              NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN              NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN              NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN              NaN

[81 rows x 4 colum

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

...

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

[81 rows x 4 columns]
         Gene ID              sequence                    Genomic Location
guide                                                                     
0           SHAM  CTTTCCTTCCCAGGGCGTGA  SHAM                            NA
1           ACTB  CTTTCCTTCCCAGGGCGTGA   NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA   NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA   NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG   NaN    chr7:76,329,203-76,331,202
...          ...                   ...   ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG   NaN                           NaN
77          ISPD  CGCCCGCAGCCATGGAGGCC   NaN                           NaN
78           ZP3  TGCTGTTCTTCTCTCAGGCA   NaN                           NaN
79         CALN1  CCAGAACCCGAAAGGCCTCT   NaN                           NaN
80        LSMEM1  TTGTTTCATATTAGCTCTAG   NaN                           NaN

[8

[81 rows x 4 columns]
         Gene ID              sequence                   Genomic Location
guide                                                                    
1           ACTB  CTTTCCTTCCCAGGGCGTGA  NaN      chr7:5,528,404-5,530,403
2      HNRNPA2B1  AGCCACCAATAAAGAGCTTA  NaN    chr7:26,196,703-26,198,702
3           SND1  CCCCTACGTAGGTCCTCTCA  NaN  chr7:127,685,612-127,687,611
4          YWHAG  TTTCGTTCCTCATTCGACAG  NaN    chr7:76,329,203-76,331,202
5        EIF2AK1  TTTAACACCTGGATTTCTGC  NaN      chr7:6,053,681-6,055,680
...          ...                   ...  ...                           ...
76      GTF2IRD1  TTGGACAGCGCTGAGCACTG  NaN    chr7:74,511,837-74,513,836
77          ISPD  CGCCCGCAGCCATGGAGGCC  NaN    chr7:16,420,324-16,422,323
78           ZP3  TGCTGTTCTTCTCTCAGGCA  NaN    chr7:76,431,920-76,433,919
79         CALN1  CCAGAACCCGAAAGGCCTCT  NaN    chr7:72,105,278-72,107,277
80        LSMEM1  TTGTTTCATATTAGCTCTAG  NaN  chr7:112,485,919-112,487,918

[80 rows x 4 co

File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
https://www.ncbi.nlm.nih.gov/tools/primer-blast/index.cgi?ORGANISM=Homo%20sapiens&INPUT_SEQUENCE=NC_000007.14&PRIMER5_START=39689379&PRIMER3_END=39691378&PRIMER_PRODUCT_MAX=1999&SHOW_SVIEWER=true

Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
https://www.ncbi.nlm.nih.gov/tools/primer-blast/index.cgi?ORGANISM=Homo%20sapiens&INPUT_SEQUENCE=NC_000007.14&PRIMER5_START=45037266&PRIMER3_END=45039265&PRIMER_PRODUCT_MAX=1999&SHOW_SVIEWER=true

Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/santiagoacero/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]
https://www.ncbi.nlm.nih.gov/tools/primer-blast/index.cgi?ORGANISM=Homo%20sapiens&INPUT_SEQUENCE=NC_000007.14&PRIMER5_START=74095652&PRIMER3_END=740

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.149)
